PER calcuation based on the last n years of op income, EBITDA and net income for a group of companies in a sector

Company value = current value + growth value
Capacity of income (cash) generation = normalization of incomes over a cycle of business

One effort is to estimate the current value based on the capacity of income generation of a company, using data from the last n years of op income, EBITDA and net income 



In [1]:
import FinanceDataReader as fdr
import pandas as pd
import OpenDartReader 
import json


In [2]:
# df_krx preparation

# df_krx_desc = fdr.StockListing('KRX-DESC')
# df_krx = fdr.StockListing('KRX')

# df_krx.drop(columns=['Close', 'ChangeCode', 'Changes', 'ChagesRatio', 'Open', 'High', 'Low', 'Volume', 'Amount'], inplace=True)
# cols_to_use = df_krx_desc.columns.difference(df_krx.columns).tolist()
# cols_to_use.append('Code')
# df_krx = df_krx.merge(df_krx_desc[cols_to_use], on='Code', how='left')

# df_krx=df_krx[~df_krx['Dept'].str.contains('관리')]   # remove companies in trouble
# df_krx.to_feather('df_krx.feather')

c:\Users\user\projects\trader\venv\Lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [ ]:
MARCAP_THRESHOLD = 3000*10**8 
SECTOR_TARGET = '특수 목적용 기계 제조업'
PRICE_START_DATE = '2013-01-01'

In [3]:
# post process of df_krx

df_krx = pd.read_feather('df_krx.feather')

df_krx = df_krx.loc[df_krx['Marcap'] >= MARCAP_THRESHOLD]
sectors_count = df_krx[['Code', 'Sector']].dropna().groupby('Sector').count().sort_values('Code', ascending=False)
sector = df_krx.loc[df_krx['Sector']==SECTOR_TARGET]


In [13]:
# sector specific prices 

# sector_prices = pd.DataFrame()
# for code in sector['Code'][:]:
#     sector_prices = pd.concat([sector_prices, fdr.DataReader(code, PRICE_START_DATE)['Close'].rename(code, inplace=True)], axis=1)
# sector_prices = sector_prices.sort_index()
# sector_prices.to_feather('sector_prices.feather')

In [21]:
sector_prices = pd.read_feather('sector_prices.feather')

In [25]:
with open('../../config/config.json', 'r') as json_file:
    config = json.load(json_file)
    dart_api = config['dart_api']

dart = OpenDartReader(dart_api)

In [ ]:
import datetime
current_year = datetime.datetime.now().year

records = []
for code in sector['Code'].values:
    record = {'code': code}
    record['nm_shares']=int(sector.loc[sector['Code']==code]['Stocks'].iloc[0])
    listed_year = sector.loc[sector['Code']==code]['ListingDate'].iloc[0].year
    if listed_year < 2015:  # Dart only serves since 2015
        listed_year = 2015
    elif listed_year == current_year: 
        continue
    rec = dart.finstate(code, bsns_year=listed_year)
    try:
        record[listed_year-2] = int(rec.loc[rec['account_nm']=='영업이익'].iloc[0,:]['bfefrmtrm_amount'].replace(',',''))
        record[listed_year-1] = int(rec.loc[rec['account_nm']=='영업이익'].iloc[0,:]['frmtrm_amount'].replace(',',''))
    except: 
        pass
    record[listed_year] = int(rec.loc[rec['account_nm']=='영업이익'].iloc[0,:]['thstrm_amount'].replace(',',''))
    for yr in range(listed_year+1, 2023):
        rec = dart.finstate(code, bsns_year=yr)
        opincome = int(rec.loc[rec['account_nm']=='영업이익'].iloc[0,:]['thstrm_amount'].replace(',',''))   # CFS/OFS (if Consolidated exists consolidated op income, otherwise standalone op income)
        record[yr] = opincome
    records.append(record)
    
